In [40]:
# Change the working directory to the project folder
# %cd "/content/drive/MyDrive/Custom_NER"

# Import required libraries and install any necessary packages
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

# Check the installed version of spaCy
spacy.__version__

# Check GPU information
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
# ! pip install -U spacy
# ! pip install spacy_transformersy
# ! pip install spacy==3.7.2
! pip uninstall spacy==3.6.1
! pip install spacy==3.7.2

     ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
     --------------------------------------- 0.0/12.1 MB 660.6 kB/s eta 0:00:19
     --------------------------------------- 0.0/12.1 MB 660.6 kB/s eta 0:00:19
     --------------------------------------- 0.1/12.1 MB 573.4 kB/s eta 0:00:21
     --------------------------------------- 0.1/12.1 MB 726.2 kB/s eta 0:00:17
      -------------------------------------- 0.2/12.1 MB 841.6 kB/s eta 0:00:15
      --------------------------------------- 0.2/12.1 MB 1.0 MB/s eta 0:00:12
     - -------------------------------------- 0.3/12.1 MB 1.0 MB/s eta 0:00:12
     - -------------------------------------- 0.4/12.1 MB 1.0 MB/s eta 0:00:12
     - -------------------------------------- 0.4/12.1 MB 1.1 MB/s eta 0:00:11
     -- ------------------------------------- 0.6/12.1 MB 1.4 MB/s eta 0:00:09
     -- ------------------------------------- 0.8/12.1 MB 1.6 MB/s eta 0:00:07
     --- ------------------------------------ 1.2/12.1


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
! pip install spacy-transformers

In [ ]:
! pip freeze > requirements.txt

In [44]:
# Load the annotated data from a JSON file
route_data = json.load(open('annotations.json','r'))

route_data

[['Mei apni inventory add kerna chahta hun',
  {'entities': [[9, 18, 'SCREEN'], [19, 22, 'ACTION']]}],
 ['I want to add my inventory',
  {'entities': [[10, 13, 'ACTION'], [17, 26, 'SCREEN']]}],
 ['I want to see subscription packages', {'entities': [[14, 26, 'SCREEN']]}],
 ['I want to upgrade my subscription', {'entities': [[21, 33, 'SCREEN']]}],
 ['mere wallet mei kitny pese hai', {'entities': [[5, 11, 'SCREEN']]}],
 ['mujhy apni listing dekhni hai', {'entities': [[11, 18, 'SCREEN']]}],
 ['mujhy listing ki analytics dekhni hai', {'entities': [[6, 13, 'SCREEN']]}],
 ['meri listing usage kitni hogye hai', {'entities': [[5, 12, 'SCREEN']]}],
 ['mere pass abhi kitny refreshes bachy hai',
  {'entities': [[21, 30, 'SCREEN']]}],
 ['mei refresh din mei kitny dafa use kersakta hun',
  {'entities': [[4, 11, 'SCREEN']]}],
 ['mujhy post banana hai', {'entities': [[6, 10, 'SCREEN']]}],
 ['mujhy broucher banana hai', {'entities': [[6, 14, 'SCREEN']]}],
 ['post kis tarah se banyga', {'entities': [[0,

In [5]:
spacy.__version__

NameError: name 'spacy' is not defined

In [47]:
# Initialize spaCy configuration files by copying from base_config to config.cfg
# !python -m spacy init fill-config /"your_file_path"/base_config.cfg /"your_file_path"/config/config.cfg
!python -m spacy init fill-config base_config.cfg config.cfg


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [48]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [50]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(route_data, test_size=0.3)

# Display the number of items in the training and testing sets
len(train), len(test)

# Open a file to log errors during annotation processing
file = open('train_file.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

# Close the error log file
file.close()

/home/aly/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 31/31 [00:00<00:00, 2755.38it/s]


In [51]:
!python -m spacy train config.cfg  --output output  --paths.train train_data.spacy  --paths.dev test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/aly/.local/lib/python3.10/site-packages/spacy/__main__.py", line 4, in <module>
    setup_cli()
  File "/home/aly/.local/lib/python3.10/site-packages/spacy/cli/_util.py", line 87, in setup_cli
    command(prog_name=COMMAND)
  File "/usr/lib/python3/dist-packages/click/core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "/home/aly/.local/lib/python3.10/site-packages/typer/core.py", line 778, in main
    return _main(
  File "/home/aly/.local/lib/python3.10/site-packages/typer/core.py", line 216, in _main
    rv = self.invoke(ctx)
  File "/usr/lib/python3/dist-packages/click/core.py", line 1659, in invoke
    return _process_result(sub_ctx.command.invoke(sub_

In [ ]:
! pip install spacy==3.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 33.6 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.2
    Uninstalling spacy-3.7.2:
      Successfully uninstalled spacy-3.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.6.0 which is incompatible.


In [1]:
# Import the spaCy library
import spacy
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Load the trained spaCy NER model from the specified path
nlp = spacy.load('/content/drive/MyDrive/ner/output/model-best/')

In [7]:
# Process the extracted text using the loaded spaCy NER model
doc = nlp("mujhe staff add karna hai")
# Iterate through the named entities (entities) recognized by the model
response={}
for ent in doc.ents:
  response[ent.label_]=ent.text
  # print(ent.text, "  ->>>>  ", ent.label_)
print(response)

{'SCREEN': 'staff', 'ACTION': 'add'}


In [59]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# df=pd.read_excel("navigation - Sheet1.xlsx")
df=pd.read_csv("navdata.csv")
print(df.tail())
input_enc = OrdinalEncoder(unknown_value=-1, handle_unknown='use_encoded_value', encoded_missing_value=-1)
output_enc = OrdinalEncoder(unknown_value=-1, handle_unknown='use_encoded_value', encoded_missing_value=-1 )
input_enc.fit(df[['Action','Screen']].values)
output_enc.fit(df[['Navigation']].values)



# X=pd.DataFrame()
# X["Action"]=df["Action"]
# X["Screen"]=df["Screen"]
# Y=df["Navigation"]
# X=X.values
# Y=Y.values
# print(X,Y)

          Action    Screen             Navigation
112       remove   package      /ManageMembership
113  unsubscribe  packages      /ManageMembership
114  unsubscribe   package      /ManageMembership
115          NaN      help  /helpAndSupportScreen
116          NaN     madad  /helpAndSupportScreen


OrdinalEncoder(encoded_missing_value=-1, handle_unknown='use_encoded_value',
               unknown_value=-1)

In [62]:
X = input_enc.transform(df[['Action','Screen']].values)
Y = output_enc.transform(df[['Navigation']].values)
print(len(X))

dt=DecisionTreeClassifier()
dt.fit(X, Y)

# predict

predicted = dt.predict(input_enc.transform([('None', 'help')]))
print(dt.score(X,Y))
predicted_label = output_enc.inverse_transform([predicted])
print(predicted_label)

117
1.0
[['/helpAndSupportScreen']]


In [61]:
import joblib
joblib.dump(dt,"DTree.pkl")
joblib.dump(input_enc,"input_encoder.pkl")
joblib.dump(output_enc,"output_encoder.pkl")

['output_encoder.pkl']